In [ ]:
# Install necessary libraries
!pip install insightface onnxruntime-gpu -q

# Imports
import os
from pathlib import Path

import cv2
import numpy as np
import onnxruntime
from insightface.app import FaceAnalysis

# ─── User-Configurable Parameters ─────────────────────────────────────────────
INPUT_FOLDER        = Path("/content/drive/MyDrive/FreeFuse_Project/Videos/Input")
OUTPUT_FOLDER       = Path("/content/drive/MyDrive/FreeFuse_Project/Videos/Blurred")
BLUR_INTENSITY      = 51       # Must be an odd integer
FACE_CONF_THRESHOLD = 0.3      # Detection confidence threshold (0.0–1.0)
# ────────────────────────────────────────────────────────────────────────────────

# Validate parameters
if BLUR_INTENSITY % 2 == 0:
    raise ValueError("BLUR_INTENSITY must be an odd integer.")

# Choose ONNX Runtime provider (GPU if available, otherwise CPU)
available_providers = onnxruntime.get_available_providers()
if "CUDAExecutionProvider" in available_providers:
    providers = ["CUDAExecutionProvider"]
    print("✅ GPU detected: using CUDAExecutionProvider.")
else:
    providers = ["CPUExecutionProvider"]
    print("ℹ️  GPU not found: falling back to CPUExecutionProvider.")

# Initialize the RetinaFace detector via InsightFace
print("🔍 Initializing RetinaFace detector…")
face_analyzer = FaceAnalysis(allowed_modules=["detection"], providers=providers)
face_analyzer.prepare(ctx_id=0, det_size=(640, 640))
print("✅ Face detector ready.")

def blur_face_region(frame: np.ndarray, bbox: tuple, ksize: int) -> None:
    """
    Apply a Gaussian blur to a rectangular region in `frame`.
    `bbox` should be (x1, y1, x2, y2). `ksize` must be odd.
    """
    x1, y1, x2, y2 = map(int, bbox)
    roi = frame[y1:y2, x1:x2]
    if roi.size > 0:
        frame[y1:y2, x1:x2] = cv2.GaussianBlur(roi, (ksize, ksize), 0)

def process_video(input_path: Path, output_path: Path) -> None:
    """
    Open `input_path`, blur detected faces above the confidence threshold,
    and write the result to `output_path`.
    """
    cap = cv2.VideoCapture(str(input_path))
    if not cap.isOpened():
        print(f"⚠️  Could not open {input_path.name}, skipping.")
        return

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    total  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    print(f"\n▶ Processing {input_path.name} → {output_path.name}")
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect faces and blur them
        for face in face_analyzer.get(frame):
            if face.det_score < FACE_CONF_THRESHOLD:
                continue
            blur_face_region(frame, face.bbox, BLUR_INTENSITY)

        writer.write(frame)
        frame_index += 1
        if frame_index % 100 == 0:
            print(f"   • processed {frame_index}/{total} frames…")

    cap.release()
    writer.release()
    print(f"✅ Finished {output_path.name}")

def main():
    # Ensure folders exist
    INPUT_FOLDER.mkdir(parents=True, exist_ok=True)
    OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

    # Process each video file
    valid_exts = {".mp4", ".mov", ".avi", ".mkv"}
    for vid in INPUT_FOLDER.iterdir():
        if vid.suffix.lower() not in valid_exts:
            continue
        out = OUTPUT_FOLDER / f"{vid.stem}_blurred{vid.suffix}"
        process_video(vid, out)

    print("\n🎉 All videos have been processed!")

if __name__ == "__main__":
    main()